In [6]:
from PIL import Image
import numpy as np

def dct_2d(matrix):
    M, N = matrix.shape
    dct = np.zeros((M, N))
    
    for u in range(M):
        for v in range(N):
            cu = np.sqrt(1/M) if u == 0 else np.sqrt(2/M)
            cv = np.sqrt(1/N) if v == 0 else np.sqrt(2/N)
            
            sum_val = 0.0
            for x in range(M):
                for y in range(N):
                    sum_val += matrix[x, y] * \
                        np.cos((2*x + 1) * u * np.pi / (2*M)) * \
                        np.cos((2*y + 1) * v * np.pi / (2*N))
            
            dct[u, v] = cu * cv * sum_val
    return dct

def phash(img, hash_size=8, highfreq_factor=4):
    img = img.convert("L").resize((hash_size * highfreq_factor,
                                   hash_size * highfreq_factor))
    pixels = np.array(img, dtype=np.float32)

    # compute DCT manually
    dct_result = dct_2d(pixels)

    # take top-left low-frequency region
    dct_low = dct_result[:hash_size, :hash_size]
    med = np.median(dct_low)
    diff = dct_low > med

    # convert to hex string
    return ''.join('%02x' % int(''.join('1' if b else '0' for b in row), 2)
                   for row in diff)

def average_hash(img, hash_size=8):
    img = img.convert("L").resize((hash_size, hash_size))
    pixels = np.array(img)
    avg = pixels.mean()
    bits = pixels > avg
    return ''.join('%02x' % int(''.join('1' if b else '0' for b in row), 2)
                   for row in bits)


# -------------------------
# Usage
image_paths = [
    "IMG_8247.jpg",
    "IMG_8248.jpg"
]
avg_results = []
p_results = []

for path in image_paths:
    img = Image.open(path)
    ah = average_hash(img)
    ph = phash(img)
    avg_results.append(ah)
    p_results.append(ph)

print("average hash comparison:")
for hash in avg_results:
    print(hash)

print("perceptual hash comparison:")
for hash in p_results:
    print(hash)

average hash comparison:
ffffe7c100000000
ffe7c7c100000078
perceptual hash comparison:
e3afc6ddc828c321
e1b7c2d7c85cc846
